## read json

In [12]:
import os 
import json
import numpy as np

name="./trace_Worker_768451.json"

def read_json(jsonPath):
#     jsonPath = os.path.join(path, dir_name)  
    jDatas=[]
    with open(jsonPath,'r') as f: 
        for line in f:
            try:
                jdata=json.loads(line.rstrip(';\n'))
#                 print(jdata)
                jDatas.append(jdata)
            except ValueError:
                print ("Skipping invalid line {0}".format(repr(line)))
                    
    return jDatas


In [25]:
item2num={"COAL":0, "COBBLESTONE":1, "CRAFTING_TABLE":2, "DIRT":3, "FURNACE":4, "IRON_AXE":5, "IRON_INGOT":6,\
         "IRON_ORE":7, "IRON_PICKAXE":8, \
          "JUNGLE_LOG":9,"OAK_LOG":9,"DARK_OAK_LOG":9,"SPRUCE_LOG":9,"ACACIA_LOG":9,"BIRCH_LOG":9,"MANGROVE_LOG":9,\
         "JUNGLE_PLANKS":10,"OAK_PLANKS":10,"DARK_OAK_PLANKS":10,"SPRUCE_PLANKS":10,"ACACIA_PLANKS":10,"BIRCH_PLANKS":10,\
         "STICK":11, "STONE":12, \
         "STONE_AXE":13, "STONE_PICKAXE":14, "TORCH":15, "WOODEN_AXE":16, "WOODEN_PICKAXE":17} #for inventoryTrace #currently no "STONE"

reward_mask=[0,16,4,0,32,0,128,64,256,1,2,4,0,0,32,0,0,8]
reward_mask=np.array(reward_mask)


In [ ]:
def extractFeatures(jDatas):    
    inventoryDiffs=[]
    inventoryLists=[]
    steps=len(jDatas)
    inventory_firstGainOrder=np.zeros((18,)) #0
    order_count=1
    inventory_firstGainStep=np.zeros((18,)) #1
    inventory_accum=np.zeros((18,)) #2.0 (return: inventory_accum_reward)
    inventory_rewardedGainStep=np.zeros((18,)) #3
    inventory_rewardedGainOrder=np.zeros((18,)) #4
    
    attack_count=0
    equip_tool_atk_count=0
    
    cmr_mov_count=0
    pos_mov_count=0
    
    #inventory0
    inventoryTrace0=jDatas[0]["inventoryTrace"]
    inventorylist0=np.zeros((18,))
    for item in inventoryTrace0.keys():
        inventorylist0[item2num[item]] = inventoryTrace0[item]
    inventoryLists.append(inventorylist0)
    
    #movings0
    directionDifference0=jDatas[0]["directionDifference"]
    playerPosition0=jDatas[0]["playerPosition"]
    
    
    for i in range(1,len(jDatas)):
        jdata=jDatas[i]
        
        #inventory
        inventoryTrace=jdata["inventoryTrace"]
#         inventorydiff={inventorydiff[key]=inventoryTrace[key]-inventoryTrace0[key] for key in inventoryTrace.keys()}
        inventorylist=np.zeros((18,))
        for item in inventoryTrace.keys():
            inventorylist[item2num[item]] = inventoryTrace[item]
        inventoryLists.append(inventorylist)
        
        inventorydiff=inventorylist-inventorylist0
        inventoryDiffs.append(inventorydiff)
        inventorylist0=inventorylist
        
        if np.any(inventorydiff>0):
            inventory_accum+=np.where(inventorydiff>0,inventorydiff,0)
            nonz_idx=np.where(inventorydiff>0)
            for idx in nonz_idx: 
                if inventory_firstGainOrder[idx]==0:
                    inventory_firstGainOrder[idx]=order_count
                    order_count+=1
                    inventory_firstGainStep[idx]=i
        
        #attack
        isAttacking=jdata["isAttacking"]   
        if isAttacking:
            attack_count+=1
            recentlyEquippedGoalItem=jdata["recentlyEquippedGoalItem"]
            if recentlyEquippedGoalItem=="WOODEN_PICKAXE" or recentlyEquippedGoalItem=="STONE_PICKAXE":
                equip_tool_atk_count+=1
            
        #movings
        directionDifference=jdata["directionDifference"]
        playerPosition=jdata["playerPosition"]
        isJumping=jdata["isJumping"]
        isSneaking=jdata["isSneaking"]
        isSprinting=jdata["isSprinting"]
        
        if directionDifference["x"]-directionDifference0["x"]!=0 \
            or directionDifference["y"]-directionDifference0["y"]!=0 \
            or directionDifference["z"]-directionDifference0["z"]!=0:
            cmr_mov_count+=1
        if playerPosition["x"]-playerPosition0["x"]!=0 \
            or playerPosition["y"]-playerPosition0["y"]!=0 \
            or playerPosition["z"]-playerPosition0["z"]!=0 \
            or isJumping or isSneaking or isSprinting:
            pos_mov_count+=1
        directionDifference0=directionDifference
        playerPosition0=playerPosition
        
        #place
    
    #(inventory) seq 0-4
    inventory_accum_reward=[inventory_accum[x]*reward_mask[x] for x in range(18)]
    inventory_accum_reward=np.array(inventory_accum_reward)
    inventory_rewardedGainStep=np.where(reward_mask>0,inventory_firstGainStep,0)
    inventory_rewardedGainOrder=np.where(reward_mask>0,inventory_firstGainOrder,0)    
    
    #(inventory) if axes are made 5-7
    if_iron_axe = inventory_accum[5]>0
    if_stone_axe= inventory_accum[13]>0
    if_wooden_axe=inventory_accum[16]>0
    
    #(inventory) reward 8-9
    sparse_reward=np.sum(np.where(inventory_accum_reward>0,reward_mask,0))
    dense_reward=np.sum(inventory_accum_reward)
    
    #(attack) 10-12
    attack_effi=(inventory_accum[1]+inventory_accum[7]+inventory_accum[9])*1.0/attack_count #total_digged_items/attack
    attack_ratio=attack_count*1.0/len(jDatas) #attack/steps
    attack_equipped=equip_tool_atk_count*1.0/attack_count #attack_and_equipped_pickaxe/attack

    #(movings) 13-14
    camera_mov=cmr_mov_count*1.0/len(jDatas)
    position_mov=pos_mov_count*1.0/len(jDatas)
    
    #(place) 15-18
    torch_placed=inventory_accum[15]-(inventoryLists[steps-1][15]-inventoryLists[0][15])
    cobblestone_placed=inventory_accum[1]-(inventoryLists[steps-1][1]-inventoryLists[0][1])
    dirt_placed=inventory_accum[3]-(inventoryLists[steps-1][3]-inventoryLists[0][3])
    stone_placed=inventory_accum[12]-(inventoryLists[steps-1][12]-inventoryLists[0][12])
    if_smelt_coal=False
    
    print("inventory_firstGainOrder:",inventory_firstGainOrder)
    print("inventory_firstGainStep:",inventory_firstGainStep)
    print("inventory_accum:",inventory_accum)
    print("inventory_accum_reward:",inventory_accum_reward)
    print("inventory_rewardedGainStep:",inventory_rewardedGainStep)
    print("inventory_rewardedGainOrder:",inventory_rewardedGainOrder)
    
    print("if_iron_axe:",if_iron_axe)
    print("if_stone_axe:",if_stone_axe)
    print("if_wooden_axe:",if_wooden_axe)
    
    print("sparse_reward:",sparse_reward)
    print("dense_reward:",dense_reward)
    
    print("attack_effi:",attack_effi)
    print("attack_ratio:",attack_ratio)
    print("attack_equipped:",attack_equipped)

    print("camera_mov:",camera_mov)
    print("position_mov:",position_mov)
    
    print("place(torch, cobblestone, dirt, stone):", torch_placed, cobblestone_placed, dirt_placed, stone_placed)
    
    return (inventory_firstGainOrder, inventory_firstGainStep,\
            inventory_accum_reward, inventory_rewardedGainStep, inventory_rewardedGainOrder,\
            if_iron_axe, if_stone_axe, if_wooden_axe, sparse_reward, dense_reward, \
            attack_effi, attack_ratio, attack_equipped, camera_mov, position_mov, \
            torch_placed, cobblestone_placed, dirt_placed, stone_placed, if_smelt_coal)



In [13]:
jDatas=read_json(name)

Skipping invalid line '{"player":"e2a8b946-590d-47a6-b223-409d018b160d","isAttacking":true,"isPlacing":true,"playerPosition":{"x":0.22458285146196283,"y":-0.27143625266615'


In [26]:
extractFeatures(jDatas)

inventory_firstGainOrder: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
inventory_firstGainStep: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
inventory_accum: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
inventory_accum_reward: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
inventory_rewardedGainStep: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
inventory_rewardedGainOrder: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
if_iron_axe: False
if_stone_axe: False
if_wooden_axe: False
sparse_reward: 0
dense_reward: 0.0
attack_effi: 0.0
attack_ratio: 0.8082191780821918
attack_equipped: 0.0
camera_mov: 0.27945205479452057
position_mov: 0.27945205479452057
place(torch, cobblestone, dirt, stone): 0.0 0.0 11.0 0.0


### Current json form: 
"isAttacking": (bool)
"isPlacing": (bool)
"isSmelting": (bool)
"playerPosition": (dic{"x":float,"y":float,"z":float})
"directionDifference": (dic{"x":float,"y":float,"z":float})
"recentlyEquippedGoalItem": (string)
"isJumping": (bool)
"isSneaking": (bool)
"isSprinting": (bool)
"inventoryTrace": (dic{"BIRCH_PLANKS":0,
                     "IRON_PICKAXE":0,
                     "COBBLESTONE":0,
                     "JUNGLE_PLANKS":0,
                     "IRON_ORE":0,
                     "WOODEN_PICKAXE":0,
                     "JUNGLE_LOG":0,
                     "DIRT":11,
                     "DARK_OAK_LOG":0,
                     "FURNACE":0,
                     "OAK_LOG":0,
                     "STONE_AXE":0,
                     "SPRUCE_LOG":0,
                     "COAL":0,
                     "OAK_PLANKS":0,
                     "ACACIA_PLANKS":0,
                     "TORCH":62,
                     "WOODEN_AXE":0,
                     "ACACIA_LOG":0,
                     "SPRUCE_PLANKS":0,
                     "IRON_INGOT":0,
                     "CRAFTING_TABLE":0,
                     "IRON_AXE":0,
                     "STONE_PICKAXE":0,
                     "DARK_OAK_PLANKS":0,
                     "STICK":0})(int)

**(currently I'm not using the last 2 keys, but it's nice to keep them.)**
"unixTime": (int)
"timestep": (int)


"isPlacing"(bool)
"isSmelting"(bool)#new added